In [1]:
import sys
sys.path.append('..')
from src.utils.cube_utils import LlamaIndexRetriever
from langchain.vectorstores import FAISS
from langchain.embeddings.azure_openai import AzureOpenAIEmbeddings
from langchain.embeddings.openai import OpenAIEmbeddings
from dotenv import load_dotenv
import os
from langchain.chat_models import AzureChatOpenAI

def load_env_variables(file_path):
    load_dotenv(file_path)
    print("Environment variables loaded successfully!")

env_file_path = "../.env"
load_env_variables = load_env_variables(env_file_path)


# embeddings = AzureOpenAIEmbeddings(azure_deployment=azure_deployment, openai_api_version=openai_api_version)
embeddings =  AzureOpenAIEmbeddings(
        deployment=os.getenv("EMB_DEPLOYMENT"),
        openai_api_version=os.getenv("EMB_OPENAI_API_VERSION"),
        model=os.getenv("EMB_MODEL"),
        openai_api_key=os.getenv("EMB_OPENAI_API_KEY"),
        openai_api_base=os.getenv("EMB_OPENAI_ENDPOINT"),
        openai_api_type=os.getenv("EMB_API_TYPE"),
    )

llm_gpt = AzureChatOpenAI(deployment_name=os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME'), openai_api_version=os.getenv("OPENAI_API_VERSION"),
                        openai_api_base=os.getenv("OPENAI_API_BASE"), 
                        openai_api_type= os.getenv("OPENAI_API_TYPE"),
                        openai_api_key=os.getenv("OPENAI_API_KEY"),
                        max_tokens=3500,
                        temperature=0.0)
# for key, value in os.environ.items():
#   print(f"{key}: {value}")

Environment variables loaded successfully!


/Users/k.abhishek/anaconda3/envs/agentgpt/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `AzureOpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureOpenAIEmbeddings`.
  warn_deprecated(
/Users/k.abhishek/anaconda3/envs/agentgpt/lib/python3.11/site-packages/langchain_community/embeddings/azure_openai.py:113: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://polarisopenai.openai.azure.com/ to https://polarisopenai.openai.azure.com//openai.
  warnings.warn(
/Users/k.abhishek/anaconda3/envs/agentgpt/lib/python3.11/site-packages/langchain_community/embeddings/azure_openai.py:1

In [2]:
from src.utils.cube_semantic_custom import CubeSemanticLoader
def fetch_cube_metadata(*args, **kwargs):
    try:
        print("In upload_metadata to vector db")
        # # Load document from Cube meta api
        loader = CubeSemanticLoader(os.getenv("CUBE_API_URL"), os.getenv("CUBE_TOKEN"), False)
        documents = loader.load()
        return documents
    except Exception as e:
        # Handle exceptions gracefully and return an error response
        print("Error in upload_metadata: " + str(e))
        return 0

def create_vector_store(documents, local_vector_store_path, *args, **kwargs):
    print("Loaded documents: " + str(documents))
    vectorstore = FAISS.from_documents(documents, embeddings)
    vectorstore.save_local(local_vector_store_path)
    print("Vector store created and saved successfully!")

def load_vector_store(vector_store_path, embeddings, *args, **kwargs):
    # Load the vector store from the local file system
    vectorstore = FAISS.load_local(vector_store_path, embeddings, allow_dangerous_deserialization=True)
    print("Vector store loaded successfully!")
    
    return vectorstore

FAISS.similarity_search_with_relevance_scores

In [9]:
vector_store_path = "/Users/k.abhishek/Documents/experiments/metric_store/metric_store_gen_ai/data/vector_store/cube_meta_faiss_index"
fetch_cube_metadata(vector_store_path, embeddings)


In upload_metadata to vector db
http://localhost:4000/cubejs-api/v1
Loading metadata from http://localhost:4000/cubejs-api/v1...
Found 16 cube data objects in metadata.
Loading only companies, because it is a cube.
Processing companies...
SD: Processing companies.count : number...
SD: Dimension values: []
SD: Processing companies.id : number...
SD: Dimension values: []
SD: Processing companies.address : string...
SD: Dimension values: []
SD: Processing companies.email : string...
SD: Dimension values: []
SD: Processing companies.name : string...
SD: Dimension values: []
SD: Processing companies.phone : string...
SD: Dimension values: []
Loading only line_items, because it is a cube.
Processing line_items...
SD: Processing line_items.count : number...
SD: Dimension values: []
SD: Processing line_items.price : number...
SD: Dimension values: []
SD: Processing line_items.quantity : number...
SD: Dimension values: []
SD: Processing line_items.id : number...
SD: Dimension values: []
SD: Pro

1

In [11]:
vectorstore = load_vector_store(vector_store_path, embeddings)

In [ ]:
#Controller
from langchain import PromptTemplate
start_goal_prompt = PromptTemplate(
    template="""
    You are an very experienced and expert manager. You are given following query "{goal}".
    The query may contain different objectives which needs an expertise to solve. Your job is to provide a list of objectives that is easier for the experts to serve the query.
    Ensure objectives are streamlined to minimize the overall number of tasks while addressing the objective effectively. 
    Some part of the query may not be relevant to the objectives. You can ignore those parts.
    The number of tasks should be strictly less than 10.
    """,
    input_variables=["goal", "language", "tools"],
)

Tools

Metric Discovery: Assuming we are trying to 3 metrics 
Agent: Get relevant 
Get 3 top documents 

Tasks: Metric Discovery 
1. Rewrite the and Extract specifics of metric information for better retrieval
2. Extract top 3 documents related to metric information. (Basic RAG)
3. get information about metrics and compile the results and return 

Tasks: Information regarding the metric
1. Check if metric exists
2. Query vector database to descrive about the metric (RAG)



In [13]:
import json
from crewai import Agent, Task
from langchain.tools import tool
#Will create below functions once the input is setup properly
# def create_vector_store

# def ingest_cube_meta():
max_number_documents = 3


@tool("Extract metric queried.")
def extract_metric_description(query, *args, **kwargs):
  """Useful to isolate metric information to be further used in the its discovery"""
  agent = Agent(
        role='Metric Isolator',
        goal=
        'Accurate Extraction of Metric name or description',
        backstory=
        "You're a Metric isolator which is responsible for extracting the metrics name from the given content.",
        llm = llm_gpt,
        allow_delegation=False)
  task = Task(
        agent=agent,
        description=
        f' The primary objective is to ensure that all the metrics are correctly isolated and extracted from the query {query}. This extracted information should be organized in a form of list.',
        expected_output="some string",
  
    )
  extracted_metrics = task.execute()

  return extracted_metrics

@tool("Extract metric queried")
def find_metric(metric_description, *args, **kwargs):
  """Useful for general user questions related to discovery, explaination, description, interpretation of metrics/measures/KPIs, tables or columns."""
  documents = fetch_cube_metadata()
  retriever = LlamaIndexRetriever('metric discovery', llm_gpt, embeddings)
  # retriever.index_document(documents)

  # rephrased_query = f"Find the metric {metric_description}"

  response_doc = retriever.query_util(metric_description, documents)
  conversational_metadata = []
  for doc in response_doc:
    meta = {'text':doc.text, 'table_metadata': doc.metadata}
    conversational_metadata.append(meta)

  agent = Agent(
        role='Data Analyst Assistant',
        goal=
        'Empower users to understand and utilize data effectively. This includes helping them discover relevant metrics, interpreting their meaning',
        backstory=
        "The primary purpose is to bridge the gap between raw data and user comprehension, fostering a data-driven culture within the organization.",
        llm = llm_gpt,
        allow_delegation=False)  
  
  task = Task(
        agent=agent,
        description=
        """ You are responding to  question {question} with answer from the Metadata provided to you as {conversational_metadata}}.
            Strictly answer the question with the information present in metadata only.
            Respond with "Sorry cannot answer your question." if the answer is not 
            present in metadata.
            """,
        expected_output="some string",
  
    )
  output = task.execute()

  return output



prompt_metric_discovery = f"You are given a metric description as input {metric_description}. Your task is to determine whether this specific metric exists within the context provided. Thoroughly analyze the metric description and provide a definitive answer on the existence of the metric, along with any relevant details or insights that support your conclusion."

In [14]:
query = "What is the most popular feature used by our paid subscribers"
extracted_metrics = extract_metric_description(query)

/Users/k.abhishek/anaconda3/envs/agentgpt/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [16]:
def get_similar_documents(query, vector_store):
  docs = vector_store.similarity_search(query)
  # take the first document as the best guess
  table_name = docs[0].metadata["table_name"]
  # Columns
  columns_question = "All available columns"
  column_docs = vector_store.similarity_search(
      columns_question,
      filter=dict(table_name=table_name),
      k=20
  )
  lines = []
  for column_doc in column_docs:
      column_title = column_doc.metadata["column_title"]
      column_name = column_doc.metadata["column_name"]
      column_data_type = column_doc.metadata["column_data_type"]
      lines.append(
          f"title: {column_title}, column name: {column_name}, datatype: {column_data_type}, member type: {column_doc.metadata['column_member_type']}"
      )
  columns = "\n".join(lines)
  print("columns :", columns)

  FAISS.similarity_search_with_relevance_scores

str

In [3]:
%pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [6]:
import nltk

# Download NLTK tokenizer data (one-time setup)
nltk.download('punkt')
nltk.download('stopwords')

def preprocess_text(text):
  # Tokenization
  tokens = nltk.word_tokenize(text.lower())
  # Stop word removal
  stopwords = nltk.corpus.stopwords.words('english')
  tokens = [token for token in tokens if token not in stopwords]
  return tokens

def reformat_question(question):
  # Define synonyms and related terms
  metrics = ["metric", "KPI", "measure"]
  sales_terms = ["sales", "revenue", "turnover"]
  central_tendency = ["average", "mean", "median"]
  
  # Preprocess the question
  preprocessed_question = preprocess_text(question)
  
  # Build the reformulated query string using question tokens
  query_string = "(" + " OR ".join(metrics) + ")"
  query_string += " AND (" + " OR ".join(sales_terms) + ")"
  query_string += " AND (" + " OR ".join(central_tendency) + ")"
  
  # Include question tokens for broader coverage
  query_string += " AND (" + " OR ".join(preprocessed_question) + ")"
  
  return query_string

# Example usage
question = "Get the metric for sales on avg"
reformulated_question = reformat_question(question)
print(f"Original question: {question}")
print(f"Reformatted question for FAISS search: {reformulated_question}")


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/k.abhishek/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/k.abhishek/nltk_data...


Original question: Get the metric for sales on avg
Reformatted question for FAISS search: (metric OR KPI OR measure) AND (sales OR revenue OR turnover) AND (average OR mean OR median) AND (get OR metric OR sales OR avg)


[nltk_data]   Unzipping corpora/stopwords.zip.
